In [1]:
# marcels code für pca (kommt aus data clean up sheet) (nicht benötigt!)


# Zeilen für Color-Coding und Legende wurden nur reinkopiert und sind noch fehlerhaft! Nacharbeiten und verstehen!

import ast
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

def residue_percentage(sequence, residues):
    """
    Berechnet den prozentualen Anteil jeder Aminosäure in der gegebenen Sequenz und gibt eine Liste seq_residues zurück,
    die jede Aminosäure als Tupel (residue, percentage) enthält.
    sequence: list, die eine CDR-Sequenz repäsentiert, und deren Einträge aus einem String pro Aminosäure besteht, z.B. ['A', 'R', 'N', ...]
    residues: list, die alle proteinogenen Aminosäuren genau einmal als String enthält
    """
    seq_residues = []
    for residue in residues:
        percentage = sequence.count(residue) / len(sequence)
        seq_residues.append((residue, percentage))
    return seq_residues

def show_feature_space(feature_space, CDR_region):#zusätzliches Argument für color-coding: antigen
    """
    Visualisiert den Feature-Space der CDR-Sequenzen mittels PCA in 3D.
    feature_space: pd.DataFrame, der die PDB-IDs in einer Spalte "pdb" und die prozentualen Anteile der Aminosäuren in den entsprechenden CDR-Sequenzen
    in je einer Spalte pro Aminosäure enthält.
    """
    X = feature_space.drop(columns=["pdb"])
    coords = PCA(n_components=3).fit_transform(X)
    
    # Color-Coding der Antigene
    #antigen_categories = antigen.astype('category')
    #color_codes = antigen_categories.cat.codes
    #categories = antigen_categories.cat.categories

    # Plotten der PCA-Koordinaten in 3D
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(coords[:, 0], coords[:, 1], coords[:, 2], s=20, alpha=0.7)#zusätzliche Argumente für color-coding: c=color_codes, cmap='tab20', 
    ax.set_xlabel("PCA 1")
    ax.set_ylabel("PCA 2")
    ax.set_zlabel("PCA 3")
    ax.set_title(f"PCA of {CDR_region} Sequences")

    # Legende
    #handles = [plt.Line2D([0], [0], marker='o', color='w', label=cat,
    #                       markerfacecolor=plt.cm.tab20(i / len(categories)), markersize=5) for i, cat in enumerate(categories)]
    #ax.legend(handles=handles, title="Antigen", bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=5, title_fontsize=5)
    #plt.tight_layout()
    plt.show()

# Bereite ab_ag_annotated vor
ab_ag_annotated = pd.read_csv('data/ab_ag_annotated.tsv', sep='\t')
ab_ag_annotated.drop_duplicates(subset=['pdb','CDR_H1','CDR_H2','CDR_H3', 'CDR_L1', 'CDR_L2', 'CDR_L3'], inplace=True) #Entferne identische Antikörper-Sequenzen mit gleichen PDB-IDs und CDR-Sequenzen
ab_ag_annotated.dropna(subset=['CDR_H1', 'CDR_H2', 'CDR_H3', 'CDR_L1', 'CDR_L2', 'CDR_L3'], inplace=True) #Entferne Einträge ohne CDR-Sequenzen
#top_antigene = ab_ag_annotated["antigen_name"].value_counts().nlargest(5).index
#ab_ag_annotated = ab_ag_annotated[ab_ag_annotated["antigen_name"].isin(top_antigene)].copy()
#antigen = ab_ag_annotated["antigen_name"]

# Schleife über die CDR-Regionen, um den Feature-Space für jede CDR-Sequenz zu erstellen und zu visualisieren:
for CDR_region in ["CDR_H1", "CDR_H2", "CDR_H3", "CDR_L1", "CDR_L2", "CDR_L3"]:
    # pd.dataframe mit den PDB-IDs und CDR-Sequenzen erstellen. Einträge sind keine Listen, sondern Strings, die Listen repräsentieren (aufgrund der .to_csv()-Methode)!
    sequences = ab_ag_annotated[["pdb", CDR_region]].copy()
    # CDR-Sequenzen im pd.dataframe sequences von Strings in Listen umwandeln. Die .items() greift auf die Index-Eintrag-Paare des pd.dataframes sequences zu.
    for i, s in sequences[CDR_region].items(): #i= Index, s = String, l= Liste
        l = ast.literal_eval(s)
        sequences.at[i, CDR_region] = l
    
    # Liste aller einzigartigen, proteinogenen Aminosäuren erstelllen, die in den CDR-Sequenzen vorkommen. Wird als Argument für residue_percentage benötigt.
    sequences_list = sequences[CDR_region].tolist()
    residues = sorted(np.unique(np.concatenate(sequences_list)))

    # Leere Matrix für den Feature-Space vorbereiten. Enthält die PDB-IDs, aber noch keine prozentualen Anteile der Aminosäuren.
    feature_space = pd.DataFrame(columns=residues, index=sequences.index)
    feature_space.insert(0, "pdb", ab_ag_annotated["pdb"])
    
    # Berechne den prozentualen Anteil jeder Aminosäure in den CDR-Sequenzen und füge sie in den Feature-Space ein.
    for i in feature_space.index:
        sequence = sequences.at[i, CDR_region]
        seq_residues = residue_percentage(sequence, residues)
        for residue, percentage in seq_residues: # Für jedes Tupel (Residue, prozentualer Anteil) in der Liste seq_residues
            feature_space.at[i, residue] = percentage # Füge den prozentualen Anteil des Residues in der CDR-Sequenz in die entsprechende Spalte ein

    show_feature_space(feature_space, CDR_region)

NameError: name 'pd' is not defined